### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.

####도쿄 

```

```


In [ ]:
# !pip install googletrans==4.0.0-rc1
# !pip install webdriver-manager
# !pip install requests

## 에어비앤비 주요 지역 탐색 

#### 1. 영어로된 일본 지역 리스트 본어로 전처리 
- 오시마랜드는 영오도 지원되나 일본어로 검색하는게 정확도가 높다 판단하여 일본어로 변경해준다 
- googletrans를 이용해 번역할 경우 정확도가 떨어진다
- 45개 지역 하드코딩으로 번역

In [11]:
from googletrans import Translator

main_district = pd.read_csv('../jieun/main_district_df2.csv')
# district_list  = main_district['neighbourhood_cleansed'].values
# translator = Translator()
# jp_district_list = list(map(lambda district: translator.translate(district, src='en', dest='ja').text.replace(' ',''),district_list))
# jp_district_list[: 10]


In [ ]:
main_district

,neighbourhood_cleansed,count
0,Shinjuku Ku,2278
1,Taito Ku,1597
2,Sumida Ku,1290
3,Toshima Ku,1002
4,Shibuya Ku,660
5,Minato Ku,409
6,Setagaya Ku,394
7,Ota Ku,360
8,Nakano Ku,316
9,Chuo Ku,290


#### Inside AIrbne 주요 지역

In [92]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

#### 2. geojosn 파일 이용
InsideAirbnb 제공 geo 파일 사용

In [7]:
import json
import pandas as pd

In [8]:
with open('../../Tokyo_Airbnb/data/neighbourhoods.geojson') as f:
    json_f = json.loads(f.read()) # json 라이브러리 이용

geo_df = pd.DataFrame(json_f)
geo_df = pd.read_json('../../Tokyo_Airbnb/data/neighbourhoods.geojson') # pd.read_json 이용


In [9]:
geojson_path ='../../Tokyo_Airbnb/data/neighbourhoods.geojson'
# gdf = gpd.read_file(geojson_path)
# gdf

with open(geojson_path) as f:
    geo_data = json.load(f)

geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[139.857803, 35.635799],
       [139.855301, 35.636868],
       [139.850296, 35.637665],
       [139.849899, 35.639267],
       [139.850494, 35.639999],
       [139.858398, 35.638134],
       [139.857895, 35.635864],
       [139.857803, 35.635799]]],
     [[[139.889999, 35.750801],
       [139.890396, 35.750401],
       [139.899796, 35.743732],
       [139.900101, 35.7356],
       [139.897095, 35.726665],
       [139.901398, 35.721466],
       [139.912109, 35.711266],
       [139.916107, 35.707733],
       [139.918701, 35.6978],
       [139.905899, 35.683731],
       [139.889099, 35.680599],
       [139.885803, 35.6768],
       [139.885895, 35.671131],
       [139.886597, 35.669868],
       [139.886993, 35.653866],
       [139.882095, 35.644001],
       [139.875504, 35.640533],
       [139.872894, 35.638535],
       [139.871796, 35.639332],
       [139.863495, 3

In [13]:
features = geo_data['features']

geo_dict = {}
for feature in features:
    coordinates = feature["geometry"]['coordinates']
    neighbourhood = feature["properties"]['neighbourhood']
    if neighbourhood in list(main_district['neighbourhood_cleansed'].values):
          lat_lng = coordinates[0][len(coordinates[0])//2]
          geo_dict[neighbourhood] = lat_lng

geo_dict

{'Edogawa Ku': [[139.857803, 35.635799],
  [139.855301, 35.636868],
  [139.850296, 35.637665],
  [139.849899, 35.639267],
  [139.850494, 35.639999],
  [139.858398, 35.638134],
  [139.857895, 35.635864],
  [139.857803, 35.635799]],
 'Katsushika Ku': [[139.846802, 35.796532],
  [139.8564, 35.790932],
  [139.858795, 35.794464],
  [139.862701, 35.794731],
  [139.871796, 35.797066],
  [139.877396, 35.7938],
  [139.871796, 35.788334],
  [139.871796, 35.784466],
  [139.876694, 35.780865],
  [139.881302, 35.782066],
  [139.889099, 35.783733],
  [139.893799, 35.782734],
  [139.893295, 35.778198],
  [139.880493, 35.771198],
  [139.880798, 35.764732],
  [139.887802, 35.753399],
  [139.888, 35.753201],
  [139.889999, 35.750801],
  [139.880493, 35.748867],
  [139.884903, 35.746601],
  [139.879807, 35.7416],
  [139.880005, 35.7402],
  [139.877594, 35.741135],
  [139.876495, 35.737064],
  [139.874802, 35.7374],
  [139.873703, 35.736866],
  [139.871796, 35.729797],
  [139.871109, 35.729065],
  [139.87

## 크롤링 
- 크롤링 메서드 생성 

In [ ]:
# 좌표값 리스트를 한 줄 만들기 
flat_coordinates = [ring for sublist in geo_json.values() for ring in sublist ]
print(flat_coordinates)

[[139.857803, 35.635799], [139.855301, 35.636868], [139.850296, 35.637665], [139.849899, 35.639267], [139.850494, 35.639999], [139.858398, 35.638134], [139.857895, 35.635864], [139.857803, 35.635799], [139.846802, 35.796532], [139.8564, 35.790932], [139.858795, 35.794464], [139.862701, 35.794731], [139.871796, 35.797066], [139.877396, 35.7938], [139.871796, 35.788334], [139.871796, 35.784466], [139.876694, 35.780865], [139.881302, 35.782066], [139.889099, 35.783733], [139.893799, 35.782734], [139.893295, 35.778198], [139.880493, 35.771198], [139.880798, 35.764732], [139.887802, 35.753399], [139.888, 35.753201], [139.889999, 35.750801], [139.880493, 35.748867], [139.884903, 35.746601], [139.879807, 35.7416], [139.880005, 35.7402], [139.877594, 35.741135], [139.876495, 35.737064], [139.874802, 35.7374], [139.873703, 35.736866], [139.871796, 35.729797], [139.871109, 35.729065], [139.870605, 35.724667], [139.868195, 35.723], [139.867203, 35.716999], [139.867706, 35.712467], [139.864807, 35

In [2]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

####  1. oshimaland 웹사이트 불러오기


In [3]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    # driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [4]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        print('fire 개수: ', len(fire_list))
        for fire in fire_list:
            try: 
                is_enabled = fire.is_enabled()
                actions = ActionChains(driver)                 
                if is_enabled:
                    # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                    # print('click OK')
                    if fire.is_displayed():
                        # print('display OK')
                        # x_offset = 200  # x 좌표를 조정할 값
                        # y_offset = 200  # y 좌표를 조정할 값
                        # driver.execute_script("arguments[0].scrollIntoView();", fire)
                        # actions.move_to_element_with_offset(fire, x_offset, y_offset)
                        # actions.click()
                        # actions.perform()

                        # 팝업창 클릭                         
                        actions = ActionChains(driver)                 
                        actions.move_to_element(fire).click().perform()

                        # 팝업 정보 가져오기 
                        # time.sleep(5)
                        WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                        info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                        info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                        info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                        tokyo_data_dict.append({
                            'district': district,
                            'date': info_date,
                            'address': info_address,
                            'content': info_content
                        })

                        time.sleep(3)

                    else:
                        pass
                        # print('display NOT')
                else:
                     pass
                    # print('click not')

            except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                # 에러 처리
                print(f"Error occurred: {e}")

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 지역 검색 

3-1 도쿄 주요 지역 검색 

In [21]:
from tqdm import tqdm, tqdm_notebook

tokyo_data_dict = []

def search_fire_distrinct(driver, jp_district_list):
    # 주요 지역 검색 
    for district in tqdm(jp_district_list):
        print(district)
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        # 검색어 clear
        region_search_tab.clear()
        # 지역 검색
        region_search_tab.send_keys(district)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, district)




3-2 도쿄 지역 위도 경도 검색

In [41]:
tokyo_data_dict = []

def search_fire_coordinate(driver, geo_dict):
    # 주요 지역 위도 경도 검색 
    for key, value in tqdm(geo_dict.items()):
       
        for lat_lng in value:
            print(key)
            region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
            region_search_tab.clear()  # 검색어 clear
            lng = lat_lng[1]
            lat = lat_lng[0]
            #경도 위도 검색
            search  = f'{lng} {lat}'
            region_search_tab.send_keys(search)
            time.sleep(2)
            # 검색 버튼 클릭 
            search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
            search_btn.send_keys(Keys.ENTER)
            # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
            time.sleep(8)
            # fire 객체 정보 수집 
            get_fire(driver,tokyo_data_dict, key)



### 크롤링 

In [43]:
from selenium import webdriver
 
url = 'https://www.oshimaland.co.jp/'

# chrome 이용 
#  WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"}
# chrome_option = webdriver.ChromeOptions()
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_option)

# firefox 이용 
driver = webdriver.Firefox()
loadWeb(driver, url)

#### 발생 에러들

1. WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No node with given id found"} : Chrome driver 문제
2. StaleElementReferenceException: Message: The element with the reference b3212e1d-3be9-4d7a-9bd2-9f736fc42e4a is stale; either its node document is not the active document, or it is no longer connected to the DOM
3. AttributeError: 'NoneType' object has no attribute 'text'
4. MoveTargetOutOfBoundsException

In [45]:
# 주요 지역명 검색 
# search_fire(driver, jp_district_list) 
# 위도 경도 검색 
# 지도를 60% 정도로 축소 뒤 수집 
search_fire_coordinate(driver, geo_dict)

driver.quit()

  0%|          | 0/46 [00:00<?, ?it/s]

Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku
fire 개수:  0
Edogawa Ku


  2%|▏         | 1/46 [01:21<1:00:45, 81.02s/it]

fire 개수:  0
Katsushika Ku
fire 개수:  14
Error occurred: Message: The element with the reference b55ff676-b155-494f-8cdc-6943fd0c8281 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 30180e09-c802-488a-baf1-24e303dd976e is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference a8e21122-b695-4932-be19-bc0376a7b3fb is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference b3f3ecf5-719c-4b02-9520-3f947e788a4f is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 1d853def-1314-42d7-95c5-adf1f2e80bed is stale; either its node document is not the active document, or it is no longe

  4%|▍         | 2/46 [46:31<19:53:50, 1627.97s/it]

...

Error occurred: Message: The element with the reference c48252f8-d714-4607-8a09-f239be100bc9 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference ab3b2301-5a4b-416c-9a0f-5d7fd90b30dc is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference c657eb92-9414-40cf-a145-2c67d1040e16 is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference 99d4045b-f589-4e13-bec2-09d2e9f2011d is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred: Message: The element with the reference b3915362-1500-43ee-8d6c-e7588db904fd is stale; either its node document is not the active document, or it is no longer connected to the DOM

Error occurred:

 20%|█▉        | 9/46 [2:28:14<10:28:29, 1019.18s/it]

Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku
fire 개수:  32
Bunkyo Ku


 20%|█▉        | 9/46 [2:34:05<10:33:28, 1027.25s/it]


KeyboardInterrupt: 

In [49]:
tokyo_data_dict

[{'district': 'Katsushika Ku',
  'date': '24年',
  'address': '東京都葛飾区西水元四丁目8-13',
  'content': '老婆がお風呂場で死亡'},
 {'district': 'Katsushika Ku',
  'date': '24年',
  'address': '東京都葛飾区西水元四丁目8-13',
  'content': '老婆がお風呂場で死亡'},
 {'district': 'Katsushika Ku',
  'date': '平成30年2月14日',
  'address': '埼玉県八潮市大瀬1654',
  'content': '火災による死亡'},
 {'district': 'Katsushika Ku',
  'date': '平成30年2月14日',
  'address': '埼玉県八潮市大瀬1654',
  'content': '火災による死亡'},
 {'district': 'Katsushika Ku',
  'date': '平成30年2月14日',
  'address': '埼玉県八潮市大瀬1654',
  'content': '火災による死亡'},
 {'district': 'Katsushika Ku',
  'date': '平成30年2月14日',
  'address': '埼玉県八潮市大瀬1654',
  'content': '火災による死亡'},
 {'district': 'Katsushika Ku',
  'date': '',
  'address': '東京都葛飾区東水元五丁目43-10',
  'content': '不詳'},
 {'district': 'Katsushika Ku',
  'date': '平成24年1月9日',
  'address': '埼玉県三郷市高州二丁目21-1',
  'content': '火災による死亡'},
 {'district': 'Katsushika Ku',
  'date': '令和3年11月21日',
  'address': '埼玉県三郷市高州一丁目348-1',
  'content': '火災による死亡'},
 {'district': 'Katsushi

In [46]:
driver.quit()


KeyboardInterrupt



### 크롤링 데이터 저장

- 230725 Fussa Shi 지역 부터 데이터 수집 필요 ( 7개의 지역만 수집했음에도 2시간 이상 소요)

```
 'Fussa Shi', 'Bunkyo Ku', 'Shinjuku Ku', 'Adachi Ku', 'Sumida Ku', 'Nerima Ku', 'Taito Ku', 'Kodaira Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Hino Shi', 'Kunitachi Shi', 'Kokubunji Shi', 'Itabashi Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Shibuya Ku', 'Toshima Ku', 'Chofu Shi', 'Suginami Ku', 'Tama Shi', 'Akishima Shi', 'Arakawa Ku', 'Machida Shi', 'Nishitokyo Shi', 'Meguro Ku', 'Fuchu Shi', 'Kita Ku', 'Tachikawa Shi', 'Hachioji Shi', 'Mitaka Shi', 'Okutama Machi', 'Musashino Shi', 'Musashimurayama Shi', 'Ome Shi', 'Akiruno Shi', 'Hamura Shi'
```

In [68]:
 # Chuo Ku 지역까지 수집 완료 
geo_dict.keys()

dict_keys(['Edogawa Ku', 'Katsushika Ku', 'Shinagawa Ku', 'Koto Ku', 'Chiyoda Ku', 'Minato Ku', 'Komae Shi', 'Chuo Ku', 'Fussa Shi', 'Bunkyo Ku', 'Shinjuku Ku', 'Adachi Ku', 'Sumida Ku', 'Nerima Ku', 'Taito Ku', 'Kodaira Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Hino Shi', 'Kunitachi Shi', 'Kokubunji Shi', 'Itabashi Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Shibuya Ku', 'Toshima Ku', 'Chofu Shi', 'Suginami Ku', 'Tama Shi', 'Akishima Shi', 'Arakawa Ku', 'Machida Shi', 'Nishitokyo Shi', 'Meguro Ku', 'Fuchu Shi', 'Kita Ku', 'Tachikawa Shi', 'Hachioji Shi', 'Mitaka Shi', 'Okutama Machi', 'Musashino Shi', 'Musashimurayama Shi', 'Ome Shi', 'Akiruno Shi', 'Hamura Shi'])

In [118]:
tokyo_df = pd.DataFrame(tokyo_data_dict)
tokyo_df = tokyo_df[tokyo_df['district']!='Fussa Shi']
tokyo_df

,district,date,address,content
0,Katsushika Ku,24年,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡
1,Katsushika Ku,24年,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡
2,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
3,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
4,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
...,...,...,...,...
1449,Chuo Ku,不明,,
1450,Chuo Ku,2018年,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死
1451,Chuo Ku,令和4年10月18日,,
1452,Chuo Ku,令和元年12月11日,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺


In [130]:
# 결측치 제거 
df_cleaned = tokyo_df.dropna(how='any', axis=0)

for column in df_cleaned.columns:
    df_cleaned = df_cleaned[df_cleaned[column] != '']

# 중복데이터 제거 
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned

,district,date,address,content
0,Katsushika Ku,24年,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡
2,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
7,Katsushika Ku,平成24年1月9日,埼玉県三郷市高州二丁目21-1,火災による死亡
8,Katsushika Ku,令和3年11月21日,埼玉県三郷市高州一丁目348-1,火災による死亡
9,Katsushika Ku,2023年,埼玉県三郷市鷹野一丁目255,部屋番号は記載しないが自殺あり
...,...,...,...,...
1431,Chuo Ku,平成30年2月20日,東京都中央区日本橋馬喰町一丁目13-8 リレント日本橋,９F 部屋番号不明 孤高死腐乱死体発見
1435,Chuo Ku,2020年1月現在,東京都千代田区東神田三丁目5-13 プレサンスロジェ千代田東神田 1002号室,告知事項あり（不動産情報より）
1450,Chuo Ku,2018年,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死
1452,Chuo Ku,令和元年12月11日,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺


In [132]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 461 entries, 0 to 1453
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   district  461 non-null    object
 1   date      461 non-null    object
 2   address   461 non-null    object
 3   content   461 non-null    object
dtypes: object(4)
memory usage: 18.0+ KB


In [134]:
# tokyo_df.to_excel('selenium_tokyo_data_geo.xlsx')
df_cleaned.to_csv('selenium_tokyo_data_geo.csv')

In [135]:
df = pd.read_csv('selenium_tokyo_data_geo.csv', index_col=0)

df

,district,date,address,content
0,Katsushika Ku,24年,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡
2,Katsushika Ku,平成30年2月14日,埼玉県八潮市大瀬1654,火災による死亡
7,Katsushika Ku,平成24年1月9日,埼玉県三郷市高州二丁目21-1,火災による死亡
8,Katsushika Ku,令和3年11月21日,埼玉県三郷市高州一丁目348-1,火災による死亡
9,Katsushika Ku,2023年,埼玉県三郷市鷹野一丁目255,部屋番号は記載しないが自殺あり
...,...,...,...,...
1431,Chuo Ku,平成30年2月20日,東京都中央区日本橋馬喰町一丁目13-8 リレント日本橋,９F 部屋番号不明 孤高死腐乱死体発見
1435,Chuo Ku,2020年1月現在,東京都千代田区東神田三丁目5-13 プレサンスロジェ千代田東神田 1002号室,告知事項あり（不動産情報より）
1450,Chuo Ku,2018年,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死
1452,Chuo Ku,令和元年12月11日,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺


#### 기존 오시마 데이터랑 중복 확인 

In [136]:
oshima_yun = pd.read_csv('../../Oshimaland_data/yunyoung/selenium_tokyo_data.csv', index_col=0)
oshima_ji = pd.read_csv('../../Oshimaland_data/jieun/oshimaland_dataset_final.csv')


In [138]:
# 결측치 제거 
oshima_yun = oshima_yun.dropna(how='any', axis=0)

for column in oshima_yun.columns:
    oshima_yun = oshima_yun[oshima_yun[column] != '']

# 중복데이터 제거 
oshima_yun = oshima_yun.drop_duplicates()
oshima_yun = oshima_yun.reindex(columns=['district', 'address', 'content', 'date'])
oshima_yun.rename(columns={'content': 'info'}, inplace=True)
oshima_yun

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,NaN,平成24年11月5日
2,新宿区,東京都新宿区西新宿一丁目3-9,NaN,令和4年
4,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,NaN,平成20年1月5日
6,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,NaN,令和3年5月11日
8,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,NaN,平成27年9月26日
...,...,...,...,...
298,羽村市,東京都羽村市緑ヶ丘五丁目1-50,NaN,2005
299,武蔵村山市,東京都羽村市羽中一丁目3-37,NaN,不明
300,昭島市,東京都武蔵村山市中央一丁目24-3ハウスオブブーケA 1階,NaN,平成24年
301,福生市,東京都昭島市田中町一丁目34-9,NaN,令和2年9月7日


In [139]:
oshima_ji_rename = oshima_ji.reindex(columns=[ 'address', 'info', 'occurred_date'])
oshima_ji_rename.rename(columns={'occurred_date': 'date'}, inplace=True)
oshima_ji_rename = oshima_ji_rename.drop_duplicates()
oshima_ji_rename.reset_index(drop=True, inplace=True)
oshima_ji_rename.dropna(inplace=True)
oshima_ji_rename = oshima_ji_rename.reindex(columns = ['district', 'address', 'info', 'date'])
oshima_ji_rename['district'] = oshima_ji_rename['address'].apply(lambda x: next((district for district in jp_district_list if district in x), x))
oshima_ji_rename.reindex(columns=['district', 'address', 'info', 'occurred_date'])

oshima_ji_rename


,district,address,info,date
0,新宿区,東京都新宿区歌舞伎町一丁目202,819号室で女性死亡,平成30年5月23日
1,新宿区,東京都新宿区歌舞伎町一丁目202アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月18日
2,新宿区,東京都新宿区西新宿七丁目1813,告知事項あり\r\n※管理会社に直接確認済み\r\n\r\nハイム大成ビル 403号室\r\...,1年以内
3,新宿区,東京都新宿区西新宿七丁目1713パレステュディオ新宿WEST,１４Ｆから女性が飛び降り自殺,令和元年9月23日
4,新宿区,東京都新宿区西新宿七丁目2231,事故死,2016
...,...,...,...,...
561,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
562,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
563,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
564,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [140]:
combined_df = pd.concat([oshima_yun, oshima_ji_rename], ignore_index=True)
combined_df = combined_df.drop_duplicates()
combined_df

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,NaN,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,NaN,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,NaN,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,NaN,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,NaN,平成27年9月26日
...,...,...,...,...
803,武蔵野市,東京都武蔵野市緑町二丁目34都営武蔵野緑町二丁目第2アパート4号棟,【青酸カリによる死者が出たそうです】\r\n4号棟の自治会役員の方の証言で、団地内で青酸カリ...,不明
804,武蔵野市,東京都武蔵野市緑町二丁目64 都営武蔵野緑町2丁目第3アパート4号棟,死体発見,平成17年12月28日
805,武蔵野市,東京都武蔵野市西久保三丁目128 アネシスヴィーゼ205号室,告知事項あり,不明
806,武蔵野市,東京都武蔵野市西久保三丁目1118,不審死,2022年11月


In [141]:
new_oshima = df.reindex(columns=['district', 'address', 'content', 'date'])
new_oshima.rename(columns={'content': 'info'}, inplace=True)
new_oshima

,district,address,info,date
0,Katsushika Ku,東京都葛飾区西水元四丁目8-13,老婆がお風呂場で死亡,24年
2,Katsushika Ku,埼玉県八潮市大瀬1654,火災による死亡,平成30年2月14日
7,Katsushika Ku,埼玉県三郷市高州二丁目21-1,火災による死亡,平成24年1月9日
8,Katsushika Ku,埼玉県三郷市高州一丁目348-1,火災による死亡,令和3年11月21日
9,Katsushika Ku,埼玉県三郷市鷹野一丁目255,部屋番号は記載しないが自殺あり,2023年
...,...,...,...,...
1431,Chuo Ku,東京都中央区日本橋馬喰町一丁目13-8 リレント日本橋,９F 部屋番号不明 孤高死腐乱死体発見,平成30年2月20日
1435,Chuo Ku,東京都千代田区東神田三丁目5-13 プレサンスロジェ千代田東神田 1002号室,告知事項あり（不動産情報より）,2020年1月現在
1450,Chuo Ku,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死,2018年
1452,Chuo Ku,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺,令和元年12月11日


In [142]:
combined_df2 = pd.concat([combined_df, new_oshima], ignore_index=True)
combined_df2 = combined_df2.drop_duplicates()
combined_df2

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,NaN,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,NaN,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,NaN,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,NaN,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,NaN,平成27年9月26日
...,...,...,...,...
1263,Chuo Ku,東京都中央区日本橋馬喰町一丁目13-8 リレント日本橋,９F 部屋番号不明 孤高死腐乱死体発見,平成30年2月20日
1264,Chuo Ku,東京都千代田区東神田三丁目5-13 プレサンスロジェ千代田東神田 1002号室,告知事項あり（不動産情報より）,2020年1月現在
1265,Chuo Ku,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死,2018年
1266,Chuo Ku,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺,令和元年12月11日


#### 중복 데이터 확인 
- 7개의 지역만 추가 트롤링 했을때 기존 807개의 데이터에서  1183개의 데이터 즉 376 개의 새로운 데이터 수집 
- 대략적으로 46개의 지역을 다 크롤링 할 시 2000개 이상의 데이터 추가 수집 가능 

In [144]:
# address 컬럼의 값이 중복되면 하나만 남기고 제거
combined_df2 = combined_df2.drop_duplicates(subset=['address'], keep='first')
combined_df2

,district,address,info,date
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,NaN,平成24年11月5日
1,新宿区,東京都新宿区西新宿一丁目3-9,NaN,令和4年
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,NaN,平成20年1月5日
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,NaN,令和3年5月11日
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,NaN,平成27年9月26日
...,...,...,...,...
1261,Chuo Ku,東京都千代田区東神田二丁目8-13 アルトコーポレーションビル,5階非常階段 40歳（推定）男性、首吊り自殺,平成23年12月25日
1263,Chuo Ku,東京都中央区日本橋馬喰町一丁目13-8 リレント日本橋,９F 部屋番号不明 孤高死腐乱死体発見,平成30年2月20日
1265,Chuo Ku,東京都中央区東日本橋二丁目25-4,ホテル利用者が病死,2018年
1266,Chuo Ku,東京都台東区柳橋一丁目1-10 ドヌール柳橋,飛び降り自殺,令和元年12月11日


#### 지도 내 줌인 줌아웃 실패 

In [ ]:
# from selenium.webdriver.common.action_chains import ActionChains
# # https://stackoverflow.com/questions/64474653/how-to-webscrape-a-canvas-element-with-python-selenium
# canvas = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > canvas.rt-root')
# action = ActionChains(driver)

# map_canvas = driver.find_element_by_css_selector("div.mk-map-view > canvas.rt-root")

# from selenium.webdriver.common.action_chains import ActionChains

# # 지도 요소 위에서 마우스를 움직임
# actions = ActionChains(driver)
# actions.move_to_element(map_canvas).click().perform()

# actions.move_by_offset(0, 200).perform() # 줌인만 가능 

# #줌인 줌아웃 반복 
# action.move_to_element(canvas).move_by_offset(0, 0).click_and_hold().perform()
# action.move_by_offset(-200,0).release().perform()

#### Request API 이용한 방법 

In [ ]:
import json
from  bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib.request as req
from tqdm import tqdm, tqdm_notebook
import time
import requests

In [ ]:
#TODO shinjuku ku test
search_region = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
search_region.send_keys(jp_district_list[20])
time.sleep(2)
# 검색 버튼 클릭 
search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
search_btn.send_keys(Keys.ENTER)

time.sleep(2)

In [ ]:
json_url = 'https://api.oshimaland.co.jp/map'
headers = {
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://www.oshimaland.co.jp"
}
data = {
    # 'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
    'keys' : ['1330021123013021',  '1330021123013023',  '1330021123013201']
}

response = requests.post(json_url, headers=headers, json=data)
json_data = response.json()

In [ ]:
from itertools import chain

origin = list(data.values())
flatten_list = list(chain(*origin))
flatten_list

In [ ]:
jsonValueList = []
for idx in range(len(flatten_list)):
    for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
        jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])

In [ ]:
getDataList2 = []
for idx in tqdm_notebook(range(len(jsonValueList))):
    json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
    url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
    code = req.urlopen(url)
    soup = BeautifulSoup(code, 'html.parser')
    json_data = json.loads(soup.text)
    getDataList2.append({
        'info' : json_data['info'],
        'address' : json_data['ad'],
        'dt' : json_data['dt'],
        'cr' : json_data['cr'],
     })